In [ ]:
import json
from mention import *
import networkx as nx

In [ ]:
def graph(deps):
    return [[d['gov'],d['dep']] for d in deps]
def matched(entity,nodes):
    for i in range(entity.start,entity.end):
        if i in nodes:return i
def shortestPath(e1,e2,graph,nodes):
    try:
        path= nx.shortest_path(graph, source=matched(e1,nodes),target=matched(e2,nodes))
    except:
        return []
    shortest = []
    for i in path:
        if e1.start<=i <e1.end:
            shortest = [i]
        elif i>=e2.start:
            shortest.append(i)
            return shortest
        else:
            shortest.append(i)
    return shortest
def stringPath(path,tokens,deps):
    res = []
    for i in range(len(path)-1):
        for d in deps:
            if d['gov'] == path[i] and d['dep'] ==path[i+1]:
       
                splited=d["type"].split("_",1)
                
                res.append(":<"+splited[0]+">:")
                for t in splited[1:]:
                    res.append(t+":")
                
                if i!=len(path)-2:
                    res[-1] += tokens[path[i+1]]
                break
            elif d['dep'] == path[i] and d['gov'] ==path[i+1]:
                d['type']="-"+d["type"]
                splited=d["type"].split("_",1)
                
                res.append(":<"+splited[0]+">:")
                for t in splited[1:]:
                    res.append(t+":")
                
               # res.append(":"+d['type']+":")
                if i!=len(path)-2:
                    res[-1] += tokens[path[i+1]]
                break
    return "".join(res)

In [ ]:
filename = '/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/test_new.json'
with open(filename,'r') as f,open('./patterns','w') as patterns:
        for line in f:
            sent = json.loads(line.strip('\r\n'))
            mentions = [Mention(int(m['start']), int(m['end']), m['labels']," ".join(sent['tokens'][m['start']:m['end']])) for m in sent['mentions']]
            if len(mentions)>1:
                x = graph(sent['dep'])
                nodes = set([l for n in x for l in n])

                g = nx.Graph(x)
                for i in range(len(mentions)-1):
                    
                    link= shortestPath(mentions[i],mentions[i+1],g,nodes)
                    if len(link)<=2 and len(link)>=2:
                      #  patterns.write("#".join(mentions[i].labels)+stringPath(link,sent['tokens'],sent['dep'])+"#".join(mentions[i+1].labels)+"\n")
                        patterns.write(stringPath(link,sent['tokens'],sent['dep'])+"\n")

In [ ]:
filename = '/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_new.json'
relation_id = dict()
label_id = dict()
with open(filename,'r') as f,open('./relation2id.txt','w') as rid,open('./entity2id.txt','w') as lid,open('./train.txt','w') as out:
        for line in f:
            sent = json.loads(line.strip('\r\n'))
            mentions = [Mention(int(m['start']), int(m['end']), m['labels']," ".join(sent['tokens'][m['start']:m['end']])) for m in sent['mentions']]
            if len(mentions)>1:
                x = graph(sent['dep'])
                nodes = set([l for n in x for l in n])

                g = nx.Graph(x)
                for i in range(len(mentions)-1):
                    
                    link= shortestPath(mentions[i],mentions[i+1],g,nodes)
                    if link == []:continue
                    for l in mentions[i].labels:
                        if l not in label_id:
                            label_id[l] = len(label_id)
                    for l in mentions[i+1].labels:
                        if l not in label_id:
                            label_id[l] = len(label_id)   
                    if len(link)<=5 and len(link)>=2:
                        rel = stringPath(link,sent['tokens'],sent['dep'])
                        if len(rel) == 0:continue
                        if rel not in relation_id:
                            relation_id[rel]=len(relation_id)
                        for l in mentions[i].labels:
                            for lr in mentions[i+1].labels:
                                out.write("%s\t[XXX]%s[YYY]\t%s\n" % (l,rel,lr))
        for k,v in label_id.iteritems():
            lid.write("%s\t%d\n" % (k,v))
        for k,v in relation_id.iteritems():
            rid.write("%s\t%d\n" % (k,v))
        lid.close()
        rid.close()
                      #  patterns.write("#".join(mentions[i].labels)+stringPath(link,sent['tokens'],sent['dep'])+"#".join(mentions[i+1].labels)+"\n")
                     #   patterns.write(stringPath(link,sent['tokens'],sent['dep'])+"\n")

In [ ]:
from gensim.models import Word2Vec

In [ ]:
entities = Word2Vec.load_word2vec_format('/Users/mayk/working/rel_extractor/baseline/Relation_Extraction/TransE/label.bin',binary=False)

In [ ]:
rels = Word2Vec.load_word2vec_format('/Users/mayk/working/rel_extractor/baseline/Relation_Extraction/TransE/rels.bin',binary=False)

In [ ]:
from numpy import linalg as LA
import numpy as np
from gensim.matutils import argsort

In [ ]:

def query(query,rel,entities,rels):
    a =rels[rel]+entities[query]
    dists = [LA.norm(a-vec) for vec in entities.syn0]
    best = argsort(dists)[0]
    if entities.index2word[best]!=query: 
        print query,rel,entities.index2word[best],dists[best]

In [ ]:
for e in entities.vocab:
    for r in rels.vocab:
        query(e,r,entities,rels)

In [ ]:
"conj_and".replace('_',":")

In [10]:
filename = '/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_new.json'
relation_id = dict()
label_id = dict()
words = set()
cnt = 0
with open(filename,'r') as f,open('./relation2id.txt','w') as rid,open('./entity2id.txt','w') as lid,open('./train.txt','w') as out:
        for line in f:
            sent = json.loads(line.strip('\r\n'))
            mentions = [Mention(int(m['start']), int(m['end']), m['labels']," ".join(sent['tokens'][m['start']:m['end']])) for m in sent['mentions']]
            if len(mentions)>1:
                x = graph(sent['dep'])
                nodes = set([l for n in x for l in n])
                g = nx.Graph(x)
                for i in range(len(mentions)-1):
                    
                    link= shortestPath(mentions[i],mentions[i+1],g,nodes)
                    if link == []:continue
                    for l in mentions[i].labels:
                        if l not in label_id:
                            label_id[l] = len(label_id)
                    for l in mentions[i+1].labels:
                        if l not in label_id:
                            label_id[l] = len(label_id)   
                    if len(link)<=2 and len(link)>=2:
                        for node in link:
                            words.add(sent['tokens'][node].lower())
                        rel = stringPath(link,sent['tokens'],sent['dep'])
                        if len(rel) == 0:continue
                        if rel not in relation_id:
                            relation_id[rel]=len(relation_id)
                        if mentions[i].name == 'It' or mentions[i+1].name == 'It' or mentions[i].name == 'The' or mentions[i+1].name == 'The':
                            continue
                        if mentions[i].name.lower() in model.vocab and mentions[i+1].name.lower() in model.vocab:
                            similarity  =model.similarity(mentions[i].name.lower(),mentions[i+1].name.lower())
                            if similarity >0.5:
                                cnt +=1
                                out.write("%s\t[XXX]%s[YYY]\t%s\n" % ("#".join(mentions[i].labels),rel,"#".join(mentions[i+1].labels)))

                        #out.write(" ".join(sent['tokens'])+"\n")

                        #out.write("%s\t%s\t[XXX]%s[YYY]\t%s\t%s\n" % ("#".join(mentions[i].labels),mentions[i].name,rel,"#".join(mentions[i+1].labels),mentions[i+1].name))
        for k,v in label_id.iteritems():
            lid.write("%s\t%d\n" % (k,v))
        for k,v in relation_id.iteritems():
            rid.write("%s\t%d\n" % (k,v))

                      #  patterns.write("#".join(mentions[i].labels)+stringPath(link,sent['tokens'],sent['dep'])+"#".join(mentions[i+1].labels)+"\n")
                     #   patterns.write(stringPath(link,sent['tokens'],sent['dep'])+"\n")

In [ ]:
with open('/Volumes/My Passport/RR_backup/data/dependency_embedding/deps.words','rb') as vectors, open('/Volumes/My Passport/RR_backup/data/dependency_embedding/shortlist','wb') as out:
    for ln in vectors:
        print ln
        if ln.split()[0] in words:
            out.write(ln)
        

In [ ]:
from gensim.models import Word2Vec
from gensim.matutils import argsort
from numpy import array,asarray
import numpy as np
model  = Word2Vec.load_word2vec_format('/Volumes/My Passport/RR_backup/data/dependency_embedding/shortlist')

In [9]:
cnt

2491